In [ ]:
import sys
import os

# Obtiene la ruta absoluta del directorio 'scripts' relativo a la ubicación del notebook
ruta_scripts = os.path.abspath(os.path.join('..', 'scripts'))

# Añade la ruta al sys.path si aún no está
if ruta_scripts not in sys.path:
    sys.path.append(ruta_scripts)

from scripts.utils import preprocess_read_reps_data

In [1]:
from scripts.utils import create_dir

ModuleNotFoundError: No module named 'scripts.utils'

In [ ]:
import pandas as pd

In [ ]:
# scripts/csv_a_sqlite.py

import csv
import sqlite3
import os

def crear_conexion(db_file):
    """Crea una conexión a la base de datos SQLite especificada por db_file.
    :param db_file: Ruta del archivo de la base de datos.
    :return: Objeto de conexión o None.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(f"Error al conectar a la base de datos: {e}")
    return conn

def crear_tabla_desde_csv(conn, csv_filepath, table_name):
    """Crea una tabla en la base de datos SQLite a partir de un archivo CSV.
    :param conn: Objeto de conexión a la base de datos.
    :param csv_filepath: Ruta del archivo CSV.
    :param table_name: Nombre de la tabla a crear.
    """
    try:
        with open(csv_filepath, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader)  # Leer la primera fila como encabezado

            # Crear la sentencia SQL para crear la tabla
            columns = ', '.join([f'"{col}" TEXT' for col in header])
            create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns});"

            cursor = conn.cursor()
            cursor.execute(create_table_sql)
            conn.commit()
            print(f"Tabla '{table_name}' creada exitosamente.")

            # Insertar los datos del CSV en la tabla
            placeholders = ', '.join(['?' for _ in header])
            insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders});"
            for row in reader:
                cursor.execute(insert_sql, row)
            conn.commit()
            print(f"Datos del archivo '{os.path.basename(csv_filepath)}' insertados en la tabla '{table_name}'.")

    except FileNotFoundError:
        print(f"Error: El archivo CSV '{csv_filepath}' no fue encontrado.")
    except sqlite3.Error as e:
        print(f"Error de SQLite al crear o insertar en la tabla '{table_name}': {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

def main():
    """Función principal para procesar los archivos CSV y crear la base de datos."""
    ruta_carpeta_csv = 'data/raw/archivos_csv_originales/'
    ruta_db = 'data/processed/base_de_datos.db'

    # Crear la carpeta 'data/processed' si no existe
    os.makedirs(os.path.dirname(ruta_db), exist_ok=True)

    conn = crear_conexion(ruta_db)
    if conn is not None:
        archivos_csv = [f for f in os.listdir(ruta_carpeta_csv) if f.endswith('.csv')]
        if not archivos_csv:
            print(f"No se encontraron archivos CSV en la carpeta '{ruta_carpeta_csv}'.")
        else:
            for archivo_csv in archivos_csv:
                ruta_completa_csv = os.path.join(ruta_carpeta_csv, archivo_csv)
                nombre_tabla = os.path.splitext(archivo_csv)[0]  # Usar el nombre del archivo sin la extensión como nombre de la tabla
                crear_tabla_desde_csv(conn, ruta_completa_csv, nombre_tabla)
        conn.close()
        print("Proceso de carga de CSV a SQLite completado.")
    else:
        print("No se pudo crear la conexión a la base de datos.")

if __name__ == '__main__':
    main()